In [1]:
!git clone https://github.com/BaseKan/optimisation_workshop_data.git

Cloning into 'optimisation_workshop_data'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


# Load Data

In [4]:
from scipy import interpolate
import pickle
import numpy as np
#from compute_temps import compute_external_temps

In [5]:
pickle_data = open("optimisation_workshop_data/input_dict.pickle","rb")
input_dict = pickle.load(pickle_data)
print(input_dict.keys())

dict_keys(['solutions', 'grids', 'distances', 'mirror_distances'])


In [ ]:
solutions = np.array(input_dict["solutions"])
grids = np.array(input_dict["grids"])
distances = np.array(input_dict["distances"])
mirror_distances = np.array(input_dict["mirror_distances"])

# Reference

In [ ]:
def compute_external_temps(solutions,grids,distances,mirror_distances):
    """
    This function computes the heating of a cable due to other cables in the environment and adds these contributions as
    a separate array "external_heat" as an instance variable to the environment cable. Moreover, it adds the heat solution
    with the external_heat to an instance variable "full_solution".

    Args:
        cable_indices [int]: a list containing the indices of the environment cables for which the heating due to
        other environment cables will be computed.

    Returns: self

    """
    external_temps = [0 for i in range(len(solutions))]
    funcs = [
        interpolate.interp1d(
            grids[i],
            solutions[i],
            fill_value="extrapolate") for i in range(len(solutions))
    ]

    for i, cable in enumerate(solutions):
        # heating from cables
        for j in [j for j in range(len(solutions)) if j != i]:
            dist = distances[i,j]
            external_temps[i] += funcs[j](dist)
        # cooling from mirror cables
        for j in range(len(solutions)):
            dist = mirror_distances[j]
            external_temps[i] -= funcs[j](dist)
    return(external_temps)

In [ ]:
%%timeit

output = compute_external_temps(solutions, grids, distances, mirror_distances)

35.6 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(output)

[62.73460739862953, 81.4266534246406, 113.06628162778529, 149.81301154348333, 171.65023936243242, 196.42114885812293, 241.17646244594033, 245.94345625064489, 251.73045307639973, 260.6160457776615, 268.7757663322127, 272.0975823309014, 282.82678446113067, 287.5297124403088, 293.82790378388864, 299.90822445008484, 305.18605131585224, 309.7314286334043, 323.2948125055413, 328.19919025616355, 333.0254087909701]


# Data

In [ ]:
max([i.size for i in solutions])

243

In [ ]:
max([i.size for i in grids])


243

In [ ]:
[i.size for i in distances]
print(distances.shape)

(21, 21)


In [ ]:
mirror_distances.shape

(21,)

# Code dissection

Old:

In [ ]:
%%timeit

funcs = [
    interpolate.interp1d(
        grids[i],
        solutions[i],
        fill_value="extrapolate") for i in range(len(solutions))
]

494 µs ± 3.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
def loop_temps(funcs, solutions, distances, mirror_distances):
    external_temps = [0 for i in range(len(solutions))]
    for i, cable in enumerate(solutions):
        # heating from cables
        for j in [j for j in range(len(solutions)) if j != i]:
            dist = distances[i,j]
            external_temps[i] += funcs[j](dist)
        # cooling from mirror cables
        for j in range(len(solutions)):
            dist = mirror_distances[j]
            external_temps[i] -= funcs[j](dist)
    return(external_temps)

In [ ]:
%%timeit

output = loop_temps(funcs, solutions, distances, mirror_distances)

36.9 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(output)

[62.73460739862953, 81.4266534246406, 113.06628162778529, 149.81301154348333, 171.65023936243242, 196.42114885812293, 241.17646244594033, 245.94345625064489, 251.73045307639973, 260.6160457776615, 268.7757663322127, 272.0975823309014, 282.82678446113067, 287.5297124403088, 293.82790378388864, 299.90822445008484, 305.18605131585224, 309.7314286334043, 323.2948125055413, 328.19919025616355, 333.0254087909701]


# Single Instruction Multiple Data 

In [ ]:
distances[0,]

array([0.        , 0.11      , 0.11      , 0.22090722, 0.18135459,
       0.28511311, 0.1580771 , 0.12581307, 0.11379336, 0.33694098,
       0.28992403, 0.3243084 , 0.19618453, 0.14875795, 0.18343644,
       0.26923664, 0.22655888, 0.26916338, 0.20245585, 0.19138685,
       0.15539925])

In [ ]:
[funcs[j](distances[1,j]) for j in range(len(solutions))]

[array(34.82423643),
 array(34.5208644),
 array(7.04637087),
 array(5.18808302),
 array(5.73967502),
 array(4.95345371),
 array(4.04626149),
 array(4.31922674),
 array(4.29756338),
 array(3.20536707),
 array(3.40195398),
 array(3.27724032),
 array(3.81202438),
 array(4.0637562),
 array(3.91248893),
 array(3.52442939),
 array(3.72875268),
 array(3.54468053),
 array(3.93948092),
 array(4.10331076),
 array(4.24643015)]

In [ ]:
zero_func = [funcs[i](0) for i in range(len(distances))]
print(zero_func)

[array(34.82423643), array(34.5208644), array(34.68684601), array(34.80523089), array(34.71182822), array(34.48028203), array(11.32908557), array(11.37244726), array(11.41818549), array(11.12996755), array(11.29087218), array(11.22021135), array(11.44226492), array(11.48705569), array(11.50919432), array(11.39588783), array(11.4754479), array(11.37113849), array(11.15750284), array(11.14499197), array(11.27582022)]


In [ ]:
mirror_funcs = np.sum([funcs[i](mirror_distances[i]) for i in range(len(mirror_distances))])
print(mirror_funcs)

33.74813255276322


In [ ]:
funcs_result = [funcs[j](distances[:j,j]) for j in range(len(distances))]
print(funcs_result)

[array([], dtype=float64), array([7.01750118]), array([7.04637087, 7.04637087]), array([5.75311928, 5.18808302, 5.92848339]), array([6.11045126, 5.73967502, 7.01922587, 7.05076237]), array([5.22996682, 4.95345371, 5.70691737, 7.01049776, 7.01049776]), array([4.69078985, 4.04626149, 3.9778201 , 3.87825862, 3.79467399,
       3.46800891]), array([5.0241939 , 4.31922674, 4.15201779, 3.86550117, 3.86718476,
       3.49035637, 6.36738196]), array([5.18189367, 4.29756338, 4.27786993, 4.11472123, 4.05792119,
       3.66146606, 6.39042469, 6.39042469]), array([3.58559251, 3.20536707, 3.39563543, 4.07539614, 3.59344275,
       3.59344275, 3.89100058, 3.70606271, 3.9042325 ]), array([3.8390623 , 3.40195398, 3.62103659, 4.38593791, 3.83606419,
       3.79422491, 4.13833087, 3.94117675, 4.18676097, 6.32651037]), array([3.66391545, 3.27724032, 3.51255214, 4.36134148, 3.77295104,
       3.81455539, 3.86452854, 3.70679221, 3.91907773, 6.291017  ,
       6.291017  ]), array([4.43103954, 3.81202438, 4.

In [ ]:
[j * zero_func[max(j-1, 0)] for j in range(len(solutions))]

[0.0,
 34.824236434442966,
 69.04172880925604,
 104.06053804048862,
 139.220923556591,
 173.55914111929775,
 206.88169215082655,
 79.30359895769041,
 90.97957811380815,
 102.76366943488114,
 111.29967545607094,
 124.1995939571282,
 134.6425361614631,
 148.74944390762965,
 160.81877967970848,
 172.63791487384287,
 182.33420522501387,
 195.08261433040482,
 204.68049280070286,
 211.9925538939503,
 222.89983948448125]

In [ ]:
sum(zero_func[:2])

69.345100839071

In [ ]:
# FINAL, CORRECT
[sum(i[j] for i in funcs_result[(1+j):21]) + sum(zero_func[:j]) - mirror_funcs for j in range(len(solutions))]

[62.73460739862953,
 81.42665342464056,
 113.06628162778523,
 149.81301154348333,
 171.65023936243247,
 196.421148858123,
 241.17646244594053,
 245.94345625064508,
 251.73045307639975,
 260.6160457776615,
 268.77576633221275,
 272.09758233090133,
 282.8267844611306,
 287.5297124403088,
 293.8279037838887,
 299.9082244500849,
 305.18605131585224,
 309.73142863340433,
 323.29481250554136,
 328.1991902561636,
 333.0254087909702]

In [ ]:
[sum(funcs_result[i,j] for i in range((1+j):21)) + j * zero_func[max(j-1, 0)] - mirror_funcs for j in range(len(solutions))]

In [ ]:
[[i,j] for i in range(j + 1, 21) for j in range(len(solutions))]

[]


In [ ]:
[[i + j for i in range(j+1, 21)] for j in range(1)]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]

In [ ]:
def compute_external_temps3(solutions,grids,distances,mirror_distances):
    """
    This function computes the heating of a cable due to other cables in the environment and adds these contributions as
    a separate array "external_heat" as an instance variable to the environment cable. Moreover, it adds the heat solution
    with the external_heat to an instance variable "full_solution".

    Args:
        cable_indices [int]: a list containing the indices of the environment cables for which the heating due to
        other environment cables will be computed.

    Returns: self

    """
    funcs = [
        interpolate.interp1d(
            grids[i],
            solutions[i],
            fill_value="extrapolate") for i in range(len(solutions))
    ]
    
    zero_func_results = [funcs[i](0) for i in range(len(solutions))]
    mirror_distances_result = np.sum([funcs[i](mirror_distances[i]) for i in range(len(mirror_distances))])
    funcs_results = [funcs[j](distances[:j,j]) for j in range(len(distances))]
    
    external_temps = [sum(i[j] for i in funcs_results[(1+j):21]) + 
                      sum(zero_func_results[:j]) - mirror_distances_result for j in range(len(solutions))]
    
    return(external_temps)

In [ ]:
%%timeit

output = compute_external_temps3(solutions,grids,distances,mirror_distances)

3.42 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print(output)

[62.73460739862953, 81.4266534246406, 113.06628162778529, 149.81301154348333, 171.65023936243242, 196.42114885812293, 241.17646244594033, 245.94345625064489, 251.73045307639973, 260.6160457776615, 268.7757663322127, 272.0975823309014, 282.82678446113067, 287.5297124403088, 293.82790378388864, 299.90822445008484, 305.18605131585224, 309.7314286334043, 323.2948125055413, 328.19919025616355, 333.0254087909701]
